# Homework Assignment 6: Model Evaluation 2
As in the previous assignments, in this homework assignment you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).

This assignment will continue to utilize a copy of the extracted feature dataset we used in Homework 5. Recall that the dataset has been processed by performing log, z-score and range scaling. We continuing to use more than one partition worth of data, so for the scaling, the mean, standard deviation, minimum, and maximum were calculated using data from both partitions so that a global scaling can be performed on each partition. 


---

## Step 1: Downloading the Data

This assignment will continue to use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB) for a training set and [Partition 2](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/TCRPUD) as a testing set. 

---

For this assignment, cleaning, transforming, and normalization of the data has been completed using both partitions to find the various minimum, maximum, standard deviation, and mean values needed to perform these operations. Recall from lecture that we should not perform these operations on each partition individually, but as a whole as there may(will) be different values for these in different partitions. 

For example, if we perform simple range scaling on each partition individually and we see a range of 0 to 100 in one partition and 0 to 10 in another. After individual scaling the values with 100 in the first would be mapped to 1 just like the values that had 10 in the second. This can cause serious performance problems in your model, so I have made sure that the normalization was treated properly for you. 

Below you will find the full partitions and `toy` sampled data from each partition, where only 20 samples from each of our 5 classes have been included in the data.  

#### Full
- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Full Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition2ExtractedFeatures.csv)

#### Toy
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition2ExtractedFeatures.csv)

Now that you have the two files, you should load each into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

---

### Evaluation Metric

As was done in Homework 5, for each of the models we evaluate in this assignmnet, you will calculate the True Skill Statistic score using the test data from Partition 2 to determine which model performs the best for classifying the positive flaring class.

    True skill statistic (TSS) = TPR + TNR - 1 = TPR - (1-TNR) = TPR - FPR

Where:

    True positive rate (TPR) = TP/(TP+FN) Also known as recall or sensitivity
    True negative rate (TNR) = TN/(TN+FP) Also known as specificity or selectivity
    False positive rate (FPR) = FP/(FP+TN) = (1-TNR) Also known as fall-out or false alarm ratio


**Recall**

    True positive (TP)
    True negative (TN)
    False positive (FP)
    False negative (FN)
    
See [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for more information.

Below is a function implemented to provide your score for each model.

---

In [1]:
%matplotlib inline
import os
import itertools
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoLars
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans

from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def calc_tss(y_true, y_predict):
    '''
    Calculates the true skill score for binary classification based on the output of the confusion
    table function
    
        Parameters:
            y_true   : A vector/list of values that represent the true class label of the data being evaluated.
            y_predict: A vector/list of values that represent the predicted class label for the data being evaluated.
    
        Returns:
            tss_value (float): A floating point value (-1.0,1.0) indicating the TSS of the input data
    '''
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    
    return tp_rate - fp_rate

---

In addition to the TSS, you will be asked to also calculate the Heidke Skill Score (HSS) to see how much better your model performs than a random forecast.  

Below is a function implemented to provide your score fore each model.

---

In [3]:
def calc_hss(y_true, y_predict):
    '''
    Calculates the Heidke Skill Score for binary classification based on the output of the confusion
    table function.
    
    The HSS measures the fractional improvement of the forecast over the standard forecast.
    The "standard forecast" is usually the number correct by chance or the proportion 
    correct by chance.
    
        Parameters:
            y_true   : A vector/list of values that represent the true class label of the data being evaluated.
            y_predict: A vector/list of values that represent the predicted class label for the data being evaluated.
    
        Returns:
            hss_value (float): A floating point value (-inf,1.0) indicating the HSS of the input data. 
                Negative values indicate that the chance forecast is better, 0 means no skill, and a perfect forecast obtains a HSS of 1.
    '''
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    #print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    P = float(TP + FN)
    N = float(FP + TN)
    numerator = 2*((TP * TN) - (FN * FP))
    denominator = P*(FN + TN) + N*(TP + FP)
    
    return numerator/denominator

---

As in the previous assignment, we will be utilizing a binary classification of our 5 class dataset. So, below is the helper function to change our class labels from the 5 class target feature to the binary target feature. The function is implemented to take a dataframe (e.g. our `abt`) and prepares it for a binary classification by merging the `X`- and `M`-class samples into one group, and the rest (`NF`, `B`, and `C`) into another group, labeled with `1`s and `0`s, respectively.

---

In [4]:
def dichotomize_X_y(data: pd.DataFrame):
    """
    dichotomizes the dataset and split it into the features (X) and the labels (y).
    
    :return: two np.ndarray objects X and y.
    """
    data_dich = data.copy()
    data_dich['lab'] = data_dich['lab'].map({'NF': 0, 'B': 0, 'C': 0, 'M': 1, 'X': 1})
    y = data_dich['lab']
    X = data_dich.drop(['lab'], axis=1)
    return X.values, y.values

---

### Reading the partitions

In [5]:
data_dir = 'data/FDS'
data_file = "toy_normalized_partition1ExtractedFeatures.csv"
data_file2 = "toy_normalized_partition2ExtractedFeatures.csv"

In [6]:
abt = pd.read_csv(os.path.join(data_dir, data_file).replace('\\', '/'))
abt2 = pd.read_csv(os.path.join(data_dir, data_file2).replace('\\', '/'))

---

### Run Feature Selection

Below you have code to perform feature selction using [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). The scoring function being used is [scikit-learn f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif).

Once feature selection is done with this one method, a set of training and testing dataframes are constructed by doing the following:

* Utilizing the `get_support` function of the feature selection object we get a mask of the features we will select from our original analytics base table DataFrame.  

* The mask of selected features is then paired with the `loc` function on our datframe containing only the descriptive features to get our selected featrues on all rows in our feature dataframe.

* The set of selected features are concatenated with our labels to construct a training dataset.

* This process was then repeated to construct the testing set.

---

In [7]:
numFeat = 20

# Split the target and descriptive features for Partition 1 into two 
# different DataFrame objects
df_labels = abt['lab'].copy()
df_feats = abt.copy().drop(['lab'], axis=1)

# Split the target and descriptive features for Partition 2 inot two
# different DataFrame Objects
df_test_labels = abt2['lab'].copy()
df_test_feats = abt2.copy().drop(['lab'], axis=1)

# Do feature selection
feats1 = SelectKBest(f_classif, k=numFeat).fit(df_feats, df_labels)

In [8]:
# Construct a training dataset from Partition 1 with only the selected descriptive 
# features and the target feature
df_selected_feats1 = df_feats.loc[:, feats1.get_support()]
df_train_set1 = pd.concat([df_labels, df_selected_feats1], axis=1)

# Construct a testing dataset from Partition 2 with only the selected descriptive
# features and the target feature
df_test_selected_feats1 = df_test_feats.loc[:, feats1.get_support()]
df_test_set1 = pd.concat([df_test_labels, df_test_selected_feats1], axis=1)
df_test_set1

,lab,TOTPOT_max,TOTPOT_median,TOTPOT_linear_weighted_average,TOTPOT_quadratic_weighted_average,TOTPOT_last_value,ABSNJZH_var,ABSNJZH_difference_of_vars,SAVNCPP_min,SAVNCPP_median,...,SAVNCPP_var,SAVNCPP_linear_weighted_average,SAVNCPP_quadratic_weighted_average,SAVNCPP_last_value,R_VALUE_mean,R_VALUE_linear_weighted_average,R_VALUE_quadratic_weighted_average,ABSNJZH_max,ABSNJZH_stddev,ABSNJZH_difference_of_means
0,B,0.888176,0.910607,0.899937,0.897118,0.889232,0.544861,0.446953,0.826002,0.875710,...,0.905469,0.877613,0.879440,0.884478,0.679219,0.679430,0.679142,0.101098,0.158041,0.066318
1,B,0.897998,0.923272,0.912721,0.910271,0.902113,0.622084,0.583644,0.884346,0.911267,...,0.921677,0.910235,0.909503,0.901160,0.694106,0.690313,0.689117,0.169857,0.216134,0.046944
2,B,0.910680,0.936391,0.925860,0.923360,0.915168,0.722335,0.669964,0.927232,0.935240,...,0.946928,0.938836,0.940307,0.942041,0.621892,0.629308,0.634254,0.373180,0.324493,0.278712
3,B,0.903096,0.928696,0.918240,0.915727,0.907472,0.585993,0.582444,0.846077,0.887077,...,0.916147,0.887299,0.887924,0.863439,0.730323,0.726870,0.725750,0.125847,0.186718,0.097641
4,B,0.878469,0.901278,0.891138,0.888651,0.881540,0.606894,0.560271,0.891314,0.908367,...,0.914248,0.907830,0.907221,0.908226,0.380567,0.324239,0.274938,0.142287,0.203227,0.174156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,X,0.944975,0.972000,0.961066,0.958503,0.950173,0.771674,0.708003,0.986181,0.987464,...,0.964465,0.987145,0.986907,0.987173,0.872821,0.869824,0.868661,0.769927,0.396334,0.346541
96,X,0.934951,0.961286,0.950650,0.948183,0.940155,0.826597,0.851072,0.966643,0.973606,...,0.972932,0.973942,0.974619,0.975578,0.918191,0.919130,0.919843,0.579843,0.495160,0.337517
97,X,0.938538,0.965245,0.954360,0.951779,0.943762,0.882727,0.831361,0.981816,0.985870,...,0.979513,0.987029,0.987651,0.990187,0.952882,0.951576,0.950977,0.862346,0.621664,0.583486
98,X,0.934400,0.959772,0.949557,0.947220,0.939333,0.859894,0.791927,0.965378,0.972216,...,0.968952,0.970228,0.969830,0.966993,0.909167,0.910553,0.910627,0.579843,0.566709,0.239493


---
### Q1 (5 points)

Using the feature selection task above as a template, you will now perform feature selection again to produce a second set of training and testing data. This time you will use the [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). 

Instead of using the [LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) that is shown in the example documentation linked above, you will be utilizing the [LassoLars](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html#sklearn.linear_model.LassoLars) as your `estimator`.  You should also set the `max_features` to the number of features we are going to select (20 features). 

**Note:** The LassoLars class is a regression model and will not work on our string class lables. So, you need to map the `NF`, `B`, `C`, `M`, `X` labels to a range between `-1` and `1` before you attempt to fit the feature selection model to them. You can try evenly spacing them, or forcing the classes that will be used as the negative class into a tight range and those that will be used as the postive class into another tight range.  Maybe this another location for you to do hyperparameter tuning?

In [9]:
numFeat = 20
abt_cpy = abt.copy()
abt2_cpy = abt2.copy()

In [10]:
#----------------------------------------------
lasso_labs = abt_cpy['lab'].map({'NF':-1, 'B':-0.5, 'C':0, 'M':0.5, 'X':1})
lasso_feats = abt_cpy.drop(['lab'], axis=1)
lasso_df_feats = SelectFromModel(max_features = numFeat, estimator = LassoLars( alpha = 0, eps =1)).fit(lasso_feats, lasso_labs)
lasso_train = lasso_feats.loc[:, lasso_df_feats.get_support()]
lasso_train_set = pd.concat([abt['lab'], lasso_train], axis = 1)
lasso_train_set
#----------------------------------------------

,lab,TOTBSQ_slope_of_longest_mono_decrease,TOTUSJZ_dderivative_mean,TOTUSJZ_gderivative_mean,SAVNCPP_dderivative_mean,MEANJZD_kurtosis,TOTFX_kurtosis,MEANPOT_slope_of_longest_mono_decrease,MEANGAM_slope_of_longest_mono_decrease,MEANALP_min,...,MEANPOT_difference_of_maxs,MEANPOT_stddev,MEANPOT_last_value,MEANPOT_slope_of_longest_mono_increase,MEANPOT_difference_of_means,MEANPOT_avg_mono_decrease_slope,MEANPOT_difference_of_stds,MEANPOT_dderivative_mean,MEANPOT_gderivative_stddev,MEANPOT_gderivative_mean
0,B,0.999990,0.979171,0.979249,0.976164,0.037609,0.036443,0.999789,0.997375,0.995406,...,0.000003,0.000023,0.000072,0.000007,0.000043,0.999974,8.113658e-06,0.351491,0.000014,0.322541
1,B,0.999980,0.980098,0.979847,0.978972,0.037284,0.033039,0.999506,0.998619,0.995912,...,0.000031,0.000073,0.000113,0.000013,0.000096,0.999944,2.846932e-05,0.351517,0.000032,0.322561
2,B,0.999849,0.979165,0.979242,0.979138,0.040767,0.040059,0.999771,0.999498,0.996573,...,0.000002,0.000017,0.000058,0.000002,0.000017,0.999989,5.385364e-06,0.351505,0.000007,0.322553
3,B,0.999559,0.977933,0.978422,0.973676,0.040877,0.035849,0.999849,0.999843,0.996432,...,0.000004,0.000018,0.000082,0.000016,0.000041,0.999983,7.232174e-06,0.351488,0.000014,0.322538
4,B,0.999913,0.979897,0.979740,0.976483,0.034652,0.027600,0.999652,0.998720,0.996480,...,0.000015,0.000051,0.000188,0.000019,0.000125,0.999952,7.075361e-07,0.351500,0.000021,0.322549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,X,0.999694,0.982453,0.981571,0.981398,0.050595,0.034746,0.999897,0.998922,0.995805,...,0.000004,0.000038,0.000259,0.000008,0.000024,0.999957,1.695746e-05,0.351499,0.000021,0.322547
96,X,0.998882,0.980923,0.980462,0.981911,0.045091,0.027407,0.999484,0.998356,0.995951,...,0.000008,0.000095,0.000351,0.000035,0.000212,0.999884,3.298372e-05,0.351538,0.000051,0.322586
97,X,0.998882,0.981685,0.981080,0.979000,0.036840,0.027419,0.998950,0.997506,0.995951,...,0.000028,0.000098,0.000339,0.000035,0.000248,0.999889,2.544730e-06,0.351536,0.000051,0.322581
98,X,0.999694,0.982395,0.981400,0.979225,0.053946,0.027424,0.999897,0.998922,0.995805,...,0.000017,0.000042,0.000253,0.000009,0.000059,0.999952,2.842715e-06,0.351485,0.000023,0.322534


In [11]:
lasso2_labs = abt2_cpy['lab'].map({'NF':-1, 'B':-0.5, 'C':0, 'M':0.5, 'X':1})
lasso2_feats = abt2_cpy.drop(['lab'], axis =1)
lasso_df_test_feats = lasso2_feats.loc[:, lasso_df_feats.get_support()]
lasso_test = lasso_feats.loc[:, lasso_df_feats.get_support()]
lasso_test_set = pd.concat([abt2['lab'], lasso_test], axis = 1)
lasso_test_set

,lab,TOTBSQ_slope_of_longest_mono_decrease,TOTUSJZ_dderivative_mean,TOTUSJZ_gderivative_mean,SAVNCPP_dderivative_mean,MEANJZD_kurtosis,TOTFX_kurtosis,MEANPOT_slope_of_longest_mono_decrease,MEANGAM_slope_of_longest_mono_decrease,MEANALP_min,...,MEANPOT_difference_of_maxs,MEANPOT_stddev,MEANPOT_last_value,MEANPOT_slope_of_longest_mono_increase,MEANPOT_difference_of_means,MEANPOT_avg_mono_decrease_slope,MEANPOT_difference_of_stds,MEANPOT_dderivative_mean,MEANPOT_gderivative_stddev,MEANPOT_gderivative_mean
0,B,0.999990,0.979171,0.979249,0.976164,0.037609,0.036443,0.999789,0.997375,0.995406,...,0.000003,0.000023,0.000072,0.000007,0.000043,0.999974,8.113658e-06,0.351491,0.000014,0.322541
1,B,0.999980,0.980098,0.979847,0.978972,0.037284,0.033039,0.999506,0.998619,0.995912,...,0.000031,0.000073,0.000113,0.000013,0.000096,0.999944,2.846932e-05,0.351517,0.000032,0.322561
2,B,0.999849,0.979165,0.979242,0.979138,0.040767,0.040059,0.999771,0.999498,0.996573,...,0.000002,0.000017,0.000058,0.000002,0.000017,0.999989,5.385364e-06,0.351505,0.000007,0.322553
3,B,0.999559,0.977933,0.978422,0.973676,0.040877,0.035849,0.999849,0.999843,0.996432,...,0.000004,0.000018,0.000082,0.000016,0.000041,0.999983,7.232174e-06,0.351488,0.000014,0.322538
4,B,0.999913,0.979897,0.979740,0.976483,0.034652,0.027600,0.999652,0.998720,0.996480,...,0.000015,0.000051,0.000188,0.000019,0.000125,0.999952,7.075361e-07,0.351500,0.000021,0.322549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,X,0.999694,0.982453,0.981571,0.981398,0.050595,0.034746,0.999897,0.998922,0.995805,...,0.000004,0.000038,0.000259,0.000008,0.000024,0.999957,1.695746e-05,0.351499,0.000021,0.322547
96,X,0.998882,0.980923,0.980462,0.981911,0.045091,0.027407,0.999484,0.998356,0.995951,...,0.000008,0.000095,0.000351,0.000035,0.000212,0.999884,3.298372e-05,0.351538,0.000051,0.322586
97,X,0.998882,0.981685,0.981080,0.979000,0.036840,0.027419,0.998950,0.997506,0.995951,...,0.000028,0.000098,0.000339,0.000035,0.000248,0.999889,2.544730e-06,0.351536,0.000051,0.322581
98,X,0.999694,0.982395,0.981400,0.979225,0.053946,0.027424,0.999897,0.998922,0.995805,...,0.000017,0.000042,0.000253,0.000009,0.000059,0.999952,2.842715e-06,0.351485,0.000023,0.322534


---
### Q2 (5 points)

In this question, you will again perform the feature selection task [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). However, this time, you will be utililizing a random forest model called [ExtraTressClssifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier) as the `estimator`. 

You need set the `max_features` of the SelectFromModel object to the number of features we are going to select (20 features). 

You also need to set the `n_estimators` of the random forest algorithm to `75` when you construct it.

**Note:** This method allows you to utilize our string class labels, so you don't need to map the lables to any other values. You can use the labels that were used in the original example above.

In [12]:
numFeat = 20

In [13]:
#----------------------------------------------
xtrees_labels = abt_cpy['lab']
xtrees_features = abt_cpy.drop(['lab'], axis = 1)
xtrees_model_features = SelectFromModel(max_features = numFeat, estimator = ExtraTreesClassifier(n_estimators = 75)).fit(xtrees_features, xtrees_labels)
xtrees_selected_features = xtrees_features.loc[:, xtrees_model_features.get_support()]
xtrees_train = pd.concat([xtrees_labels, xtrees_selected_features], axis = 1)
xtrees_train
#----------------------------------------------

,lab,TOTBSQ_linear_weighted_average,TOTPOT_mean,ABSNJZH_var,SAVNCPP_min,SAVNCPP_median,SAVNCPP_var,SAVNCPP_difference_of_medians,SAVNCPP_quadratic_weighted_average,SAVNCPP_last_value,...,USFLUX_skewness,TOTFX_kurtosis,R_VALUE_mean,R_VALUE_last_value,TOTUSJH_min,ABSNJZH_stddev,ABSNJZH_difference_of_maxs,MEANSHR_linear_weighted_average,MEANGBH_average_absolute_derivative_change,SAVNCPP_max
0,B,0.857535,0.888655,0.600188,0.867919,0.893334,0.901466,0.886079,0.890066,0.882146,...,0.466460,0.036443,0.535726,0.497707,0.145829,0.197777,0.060871,0.707192,0.294229,0.027841
1,B,0.843869,0.882247,0.680944,0.825839,0.897194,0.928598,0.933825,0.903081,0.907012,...,0.471575,0.033039,0.458838,0.593018,0.101915,0.274373,0.230947,0.843157,0.442534,0.036982
2,B,0.906763,0.911793,0.625489,0.895776,0.911852,0.925663,0.911748,0.914137,0.920578,...,0.526223,0.040059,0.575241,0.497019,0.279344,0.219138,0.074247,0.688024,0.177295,0.059151
3,B,0.944800,0.932182,0.541169,0.924288,0.931340,0.929003,0.918959,0.928458,0.921168,...,0.495887,0.035849,0.879914,0.890073,0.379392,0.155693,0.066391,0.674857,0.168761,0.095279
4,B,0.926448,0.929374,0.593376,0.813905,0.890793,0.920634,0.911303,0.888427,0.897998,...,0.439050,0.027600,0.802363,0.780744,0.317649,0.192390,0.138257,0.857433,0.185685,0.037962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,X,0.967341,0.948648,0.817282,0.971766,0.976454,0.961077,0.950802,0.976761,0.975462,...,0.490344,0.034746,0.921229,0.927977,0.518692,0.476814,0.365403,0.849314,0.171691,0.429985
96,X,0.986462,0.960412,0.981117,0.903731,0.956255,0.987593,0.949146,0.956666,0.960427,...,0.425749,0.027407,0.976325,0.959694,0.733205,0.926316,0.531832,0.849787,0.212918,0.320873
97,X,0.985814,0.959869,0.979179,0.903731,0.954739,0.988355,0.904570,0.954578,0.958772,...,0.451961,0.027419,0.979155,0.962338,0.733205,0.919066,0.531832,0.847687,0.213173,0.320873
98,X,0.966694,0.948479,0.802219,0.971766,0.976107,0.961561,0.956618,0.976678,0.975806,...,0.511945,0.027424,0.920955,0.930951,0.513776,0.448571,0.350938,0.850064,0.180257,0.429985


In [14]:
xtrees_test_labels = abt2_cpy['lab']
xtrees_test_features = abt2_cpy.drop(['lab'], axis = 1)
xtrees_test_selected = xtrees_test_features.loc[:, xtrees_model_features.get_support()]
xtrees_test = pd.concat([xtrees_test_labels, xtrees_test_selected], axis = 1)
xtrees_test

,lab,TOTBSQ_linear_weighted_average,TOTPOT_mean,ABSNJZH_var,SAVNCPP_min,SAVNCPP_median,SAVNCPP_var,SAVNCPP_difference_of_medians,SAVNCPP_quadratic_weighted_average,SAVNCPP_last_value,...,USFLUX_skewness,TOTFX_kurtosis,R_VALUE_mean,R_VALUE_last_value,TOTUSJH_min,ABSNJZH_stddev,ABSNJZH_difference_of_maxs,MEANSHR_linear_weighted_average,MEANGBH_average_absolute_derivative_change,SAVNCPP_max
0,B,0.891868,0.904676,0.544861,0.826002,0.875710,0.905469,0.871146,0.879440,0.884478,...,0.487745,0.028188,0.679219,0.677843,0.212863,0.158041,0.024713,0.649360,0.146881,0.021409
1,B,0.907375,0.916737,0.622084,0.884346,0.911267,0.921677,0.852440,0.909503,0.901160,...,0.441909,0.027338,0.694106,0.682567,0.277444,0.216134,0.056961,0.722890,0.148309,0.047650
2,B,0.930710,0.929911,0.722335,0.927232,0.935240,0.946928,0.944892,0.940307,0.942041,...,0.465092,0.029043,0.621892,0.631455,0.365649,0.324493,0.287077,0.828676,0.183252,0.123288
3,B,0.916118,0.922231,0.585993,0.846077,0.887077,0.916147,0.897691,0.887924,0.863439,...,0.454938,0.033735,0.730323,0.721790,0.269404,0.186718,0.087503,0.757868,0.149620,0.027869
4,B,0.879534,0.895181,0.606894,0.891314,0.908367,0.914248,0.757929,0.907221,0.908226,...,0.536629,0.038919,0.380567,0.000000,0.229202,0.203227,0.120954,0.652698,0.212713,0.045467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,X,0.995461,0.965111,0.771674,0.986181,0.987464,0.964465,0.934043,0.986907,0.987173,...,0.621232,0.030347,0.872821,0.871368,0.885666,0.396334,0.326953,0.881194,0.185888,0.682312
96,X,0.977947,0.954518,0.826597,0.966643,0.973606,0.972932,0.968006,0.974619,0.975578,...,0.412339,0.033952,0.918191,0.917902,0.634410,0.495160,0.228014,0.781405,0.141181,0.405530
97,X,0.984968,0.958541,0.882727,0.981816,0.985870,0.979513,0.976734,0.987651,0.990187,...,0.508683,0.046716,0.952882,0.948928,0.870969,0.621664,0.574351,0.812522,0.132968,0.658544
98,X,0.975771,0.953294,0.859894,0.965378,0.972216,0.968952,0.952973,0.969830,0.966993,...,0.521756,0.034015,0.909167,0.906332,0.629267,0.566709,0.132315,0.784807,0.138522,0.386208


---
### Q3 (5 points)

Now that you have three different datasets, you need to convert them each to a binary classification problem datase or dichotomize the training and testing data. Lucky for you, a method has already been provided to do this. All you need to do is apply it to teach of the `DataFrame`s you constructed with the feature selected training and testing data from the exmpale, Q1, and Q2.

**Note:** You might want to put the training and testing tuples you get from the call to the dichotomize method into seperate training and testing lists. Then you can loop over them later. 

In [15]:
#----------------------------------------------
train = [df_train_set1.copy(), lasso_train_set.copy(), xtrees_train.copy()]
test = [df_test_set1.copy(), lasso_test_set.copy(), xtrees_test.copy()]
di_train_x1 = []
di_train_y1 = []
di_test_x2 = []
di_test_y2 = []

for set1 in train:
    x1, y1 = dichotomize_X_y(set1)
    di_train_x1.append(x1)
    di_train_y1.append(y1)

for set2 in test:
    x2, y2 = dichotomize_X_y(set2)
    di_test_x2.append(x2)
    di_test_y2.append(y2)
#----------------------------------------------

---
### Q4 (10 points)

Now that you have your data setup, you will be constructing an [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model on each one of the different datasets. For this exercise default regularization parameter `C` value of 1.0, the default `kernel` type of `rbf`, and the default setting of the kernel coefficient `gamma` for the `rbf` kernel. You should, however, set the `class_weight` to `balanced` when you construct your models. This way the regularization parameter is adjusted for each class in proportion the occurrence of that class in the dataset.

You should train the model on your training data, then test it on the testing data with the same set of selected descriptive features. You will then calculate both the TSS and HSS scores and print them out.

**Note:** for more information on what the `C` and `gamma` parameters do on the `rbf` kernel see the [RBF SVM parameters](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html) documentation. We won't be tuning these values in this question, but it is genearally accepted that tuning should be done to find the best performing model. 

In [16]:
selected_labels = ['F-Val', 'FromLasso', 'FromForest']

In [17]:
#----------------------------------------------
for ind in range(len(selected_labels)):
     classifier = SVC(class_weight = 'balanced')
     classifier.fit(di_train_x1[ind],di_train_y1[ind])
     y_pred = classifier.predict(di_test_x2[ind])
     
     tss_score = calc_tss(di_test_y2[ind], y_pred)
     hss_score = calc_hss(di_test_y2[ind], y_pred)

     print(selected_labels[ind])
     print(f"TSS: {tss_score}")
     print(f"HSS: {hss_score}")
#----------------------------------------------

TN=46	FP=14	FN=1	TP=39
F-Val
TSS: 0.7416666666666667
HSS: 0.7035573122529645
TN=60	FP=0	FN=40	TP=0
FromLasso
TSS: 0
HSS: 0.0
TN=44	FP=16	FN=1	TP=39
FromForest
TSS: 0.7083333333333333
HSS: 0.6666666666666666


---
### Q5 (15 points)

After training and testing the SVC model on the above dataset, you will likely see that this process is quite time consuming. This is because the algorithm needs evaluate all the instances in the training dataset to find instances that can be used as points in a separating hyperplane between the samples of different classes.  

In order to speed this process, lets reduce the number of samples in the dataset through undersampling the classes like we did before. Unlike was done before, where we just pick some random sample of instances in the various classes, we will be performing some data informed under sampling using the [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) clustering algorithm.

So, what I want you to do is construct a function (I have started it below) that performs the following:

* Groups the data by the `lab` column and finds the size of the smallest group.

* For every group of samples in the dataset that is not the smallest group, you will use the KMeans algorithm to cluster the samples into `K` clusters. The size of `K` is the size of the smallest group.  **Note:** You need to make sure you are only using the descriptive features when doing this, so drop the label column from each group.

* Once you have the `K` clusters, you will get the cluster centers by using the `cluster_centers_` attribute of your Kmeans object. This attribute is a `(n_clusters, n_features)` array of values. These will be the new set of samples of descriptive features for the class you are processing. You should construct a DataFrame with these and add a label column with your class label for each one of these samples.

* The samples for the smalles class group from the original dataset will be the samples you return for that class. 

* You will need to concatenate all of the results into one DataFrame and return it at the end of the function.

Once you have completed the function, you need to apply it to each of your three training sets (the ones that have not had the dicotimize process applied, I hope you kept a copy). Then you will apply the dicotomize process to the sampled training sets and place them into a list for use in the next problem.

**Note:** By training our models on representations of the real data instead of the acutal measurements, we are building a type of surrogate model. By doing so, we can approximate how our model might behave when trained with the true data, but can test several different settings much faster than what we otherwise would.

In [18]:
def perform_under_sample_clust(data:DataFrame)->DataFrame:
    #----------------------------------------------
    temp = data.copy()
    res = pd.DataFrame()

    freq = pd.DataFrame(temp['lab'].value_counts())
    indices = freq.index.tolist()
    min_val_ind = freq.idxmin()[0]
    indices.remove(min_val_ind)
    min_v = freq['lab'][min_val_ind]

    desc_feats = temp.columns[1:]

    res = pd.concat([res, temp[temp['lab'] == min_val_ind]])

    for ind in indices:
        classifier = KMeans(n_clusters=min_v)
        classifier.fit(temp[temp['lab'] == ind].drop(['lab'], axis = 1))
        temp2 = pd.DataFrame(classifier.cluster_centers_, columns=desc_feats)
        temp2['lab'] = ind
        res = pd.concat([res, temp2])
    
    res.reset_index(inplace=True, drop=True)
    return res
    #----------------------------------------------

In [19]:
#----------------------------------------------
for frame in train:
     frame = perform_under_sample_clust(frame)

us_di_train_x1 = []
us_di_train_y1 = []

for frame in train:
     x1, y1 = dichotomize_X_y(frame)
     us_di_train_x1.append(x1)
     us_di_train_y1.append(y1)
#----------------------------------------------

---
### 6 (10 points)

In question 5 we produced datasets that approxumate what our real data looks like. By training our models on these representations of the real data instead of the acutal measurements, we are building a type of surrogate model. In doing so, we can approximate how our model might behave when trained with the true data, but we obtain a major advantage in that we can test several different settings much faster than what we otherwise would if using the true dataset. We can then use these surrogate results to find a range of the hyperparameters that we might wish to investigate using the true input data.

For this question, you will again train your models on the three different feature selected data. However, instead of the full partition 1 training datasets, you will be using the sampling with KMeans training datasets you constructed in Q5. 

You will again be constructing an [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model on each one of the different datasets. Like before, you should set the `class_weight` to `balanced` when you construct your models. Unlike the previous question where you traind the models, this time you will be asked to evaluate several different settings for the `kernel`, the regularization parameter `C`, and kernel coefficient `gamma`. **Note:** The `gamma` paramter is only utilized on the ‘rbf’, ‘poly’ and ‘sigmoid’ kernels, so there is no reason to evaluate multiple settings for the `linear` kernel. I have listed the settings of each parameter in a code block below. 

For each of the settings, you should train the model on your training data, then test it on the testing data with the same set of selected descriptive features. You will then calculate both the TSS and HSS scores and print them out.

In [20]:
selected_labels = ['F-Val', 'FromLasso', 'FromForest']

In [21]:
kernel = ['linear', 'poly', 'rbf']
c_vals = [ 0.5, 1.0]
gamma_vals = [0.5, 1, 10]
temp = [kernel, c_vals, gamma_vals]
params = list(itertools.product(*temp))

In [22]:
#----------------------------------------------
for ind in range(len(selected_labels)):
     print(selected_labels[ind], "*-*-*"*5)
     for kernel, c_value, gamma_value in params:
         classifier = SVC(class_weight = 'balanced', gamma=gamma_value, C=c_value, kernel=kernel)
         classifier.fit(us_di_train_x1[ind],di_train_y1[ind])
         y_pred = classifier.predict(di_test_x2[ind])
         tss_score = calc_tss(di_test_y2[ind], y_pred)
         hss_score = calc_hss(di_test_y2[ind], y_pred)
         print(f"TSS: {tss_score}")
         print(f"HSS: {hss_score}")
#----------------------------------------------

F-Val *-*-**-*-**-*-**-*-**-*-*
TN=48	FP=12	FN=2	TP=38
TSS: 0.75
HSS: 0.72
TN=48	FP=12	FN=2	TP=38
TSS: 0.75
HSS: 0.72
TN=48	FP=12	FN=2	TP=38
TSS: 0.75
HSS: 0.72
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=51	FP=9	FN=1	TP=39
TSS: 0.825
HSS: 0.7983870967741935
TN=51	FP=9	FN=1	TP=39
TSS: 0.825
HSS: 0.7983870967741935
TN=50	FP=10	FN=20	TP=20
TSS: 0.33333333333333337
HSS: 0.34782608695652173
TN=51	FP=9	FN=1	TP=39
TSS: 0.825
HSS: 0.7983870967741935
TN=49	FP=11	FN=1	TP=39
TSS: 0.7916666666666666
HSS: 0.76
TN=49	FP=11	FN=20	TP=20
TSS: 0.31666666666666665
HSS: 0.329004329004329
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=46	FP=14	FN=1	TP=39
TSS: 0.7416666666666667
HSS: 0.7035573122529645
TN=49	FP=11	FN=1	TP=39
TSS: 0.7916666666666666
HSS: 0.76
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0

---
### 7 (10 points)

Results above were able to find some combinations of hyperparamters and datasets that work fairly well for our problem. But the question remains, can we do better?

Maybe one way to improve our results would be to elinate the easy to classify instances from our dataset and only focus our efforts on the more difficult ones. If you recall from our data preparation there was a feature in our dataset that we could use to easily distinguish between a rather large percentage of `flare` and `non-flare` data. This feature was `R_VALUE_median`, but we don't know what value to use to filter off part of our data.

So, for this question, let's plot and see where a good cutoff might be. To do this, let's use the seaborn [ecdfplot](https://seaborn.pydata.org/generated/seaborn.ecdfplot.html#seaborn.ecdfplot) or the cumulative distribution function plot.  Your input will be the original analytics base table of partition one.  You should set the `x` axis to `R_VALUE_median`, and set the `hue` to `lab`.

After plotting this, you will see that around 0.5 we begin to see some instances of the `M` and around 0.7 we begin to see some instances of the `X` class flares in our dataset. So, use 0.5 as a threshold value to filter out all of the instances that fall below this threshold from our training data. Construct a copy of the original partition 1 data with this applied.

You can then verify this using the seaborn [kdeplot](https://seaborn.pydata.org/generated/seaborn.kdeplot.html#seaborn.kdeplot) using the new filtered data as input, setting `x` to `R_VALUE_median` again, and setting `hue` to `lab` again.  

<AxesSubplot:xlabel='R_VALUE_median', ylabel='Proportion'>

In [23]:
#----------------------------------------------
sns.ecdfplot(abt, hue='lab', x='R_VALUE_median')
#----------------------------------------------

In [24]:
filter_abt = abt[abt['R_VALUE_median'] >= 0.5].copy()

---
### 8 (10 points)

For this question, you will utilize the filtered analytics base table you constructed in the previous question.  You should:

* Repeat the feature selection I did for you in the  example using the [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection), and scoring function [scikit-learn f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif).

* Repeat the feature selection from Q1 using the [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection), and utilizing the [LassoLars](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html#sklearn.linear_model.LassoLars) as your `estimator`.  You should also set the `max_features` to the number of features we are going to select (20 features).

* Repeat the feature selection from Q2 using [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). With a random forest model called [ExtraTressClssifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier) as the `estimator`. The `n_estimators` of the random forest algorithm should be set to `75` when you construct it. The `max_features` of the SelectFromModel should be set to the number of features we are going to select (20 features).

In [25]:
numFeat = 20

# Split the target and descriptive features for Partition 1 into two 
# different DataFrame objects
df_labels = filter_abt['lab'].copy()
df_feats = filter_abt.copy().drop(['lab'], axis=1)

# Split the target and descriptive features for Partition 2 inot two
# different DataFrame Objects
df_test_labels = abt2['lab'].copy()
df_test_feats = abt2.copy().drop(['lab'], axis=1)

# Do feature selection
feats1 = SelectKBest(f_classif, k=numFeat).fit(df_feats, df_labels)
# Construct a training dataset from Partition 1 with only the selected descriptive 
# features and the target feature
df_selected_feats1 = df_feats.loc[:, feats1.get_support()]
df_train_set1 = pd.concat([df_labels, df_selected_feats1], axis=1)

# Construct a testing dataset from Partition 2 with only the selected descriptive
# features and the target feature
df_test_selected_feats1 = df_test_feats.loc[:, feats1.get_support()]
df_test_set1 = pd.concat([df_test_labels, df_test_selected_feats1], axis=1)
#----------------------------------------------
lasso_labs = filter_abt['lab'].map({'NF':-1, 'B':-0.5, 'C':0, 'M':0.5, 'X':1})
lasso_feats = filter_abt.drop(['lab'], axis =1)
lasso_df_feats = SelectFromModel(max_features = numFeat, estimator = LassoLars( alpha = 0, eps =1)).fit(lasso_feats, lasso_labs)
lasso_train = lasso_feats.loc[:, lasso_df_feats.get_support()]
lasso_train_set = pd.concat([filter_abt['lab'], lasso_train], axis = 1)
#----------------------------------------------
lasso2_labs = abt2_cpy['lab'].map({'NF':-1, 'B':-0.5, 'C':0, 'M':0.5, 'X':1})
lasso2_feats = abt2_cpy.drop(['lab'], axis =1)
lasso_df_test_feats = lasso2_feats.loc[:, lasso_df_feats.get_support()]
lasso_test = lasso_feats.loc[:, lasso_df_feats.get_support()]
lasso_test_set = pd.concat([abt2['lab'], lasso_test], axis = 1)
#----------------------------------------------
xtrees_labels = filter_abt['lab'].copy()
xtrees_features = filter_abt.copy().drop(['lab'], axis = 1)
xtrees_model_features = SelectFromModel(max_features = numFeat, estimator = ExtraTreesClassifier(n_estimators = 75)).fit(xtrees_features, xtrees_labels)
xtrees_selected_features = xtrees_features.loc[:, xtrees_model_features.get_support()]
xtrees_train = pd.concat([xtrees_labels, xtrees_selected_features], axis = 1)
#----------------------------------------------
xtrees_test_labels = abt2_cpy['lab']
xtrees_test_features = abt2_cpy.drop(['lab'], axis = 1)
xtrees_test_selected = xtrees_test_features.loc[:, xtrees_model_features.get_support()]
xtrees_test = pd.concat([xtrees_test_labels, xtrees_test_selected], axis = 1)

C:\Users\sthelluri1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [435] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\sthelluri1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


---
### Q9 (10 points)

Using the training and testing datsets you constructed in the previous question after performing feature selection on the filtered partition 1 analytics base table. You now need to perform the sampling on the training data using the function you made in Q5.

Then you should convert each of the new training and testing datasets to a binary classification problem datase or dichotomize the training and testing data like you did in Q3. Lucky for you, a method has already been provided to do this. All you need to do is apply it to teach of the `DataFrame`s you constructed with the feature selected training and testing data.

**Note:** You might want to put the training and testing tuples you get from the call to the dichotomize method into seperate training and testing lists. Then you can loop over them later. 


In [26]:
#----------------------------------------------
train = [df_train_set1.copy(), lasso_train_set.copy(), xtrees_train.copy()]
test = [df_test_set1.copy(), lasso_test_set.copy(), xtrees_test.copy()]

#undersampling
for frame in train:
     frame = perform_under_sample_clust(frame)

di_test_x1 = []
di_test_y1 = []
di_train_x2 = []
di_train_y2 = []

for frame in train:
     x1, y1 = dichotomize_X_y(frame)
     di_train_x1.append(x1)
     di_train_y1.append(y1)
    

for frame in test:
     x2, y2 = dichotomize_X_y(frame)
     di_test_x2.append(x2)
     di_test_y2.append(y2)
#----------------------------------------------

---
### Q10 (20 points)

Like in Q6, this question will be utilizing the filtered and sampled datasets constructed in the previous question. For this question, you will again train your models on the three different feature selected data that had the instances below our thrshold filtered out and then had sampling by clustering performed on them. 

You will again be constructing an [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model on each one of the different datasets. Like before, you should set the `class_weight` to `balanced` when you construct your models. You will again evaluate several different settings for the `kernel`, the regularization parameter `C`, and kernel coefficient `gamma`. **Note:** The `gamma` paramter is only utilized on the ‘rbf’, ‘poly’ and ‘sigmoid’ kernels, so there is no reason to evaluate multiple settings for the `linear` kernel. I have listed the settings of each parameter in a code block below. 

For each of the settings, you should train the model on your training data, then test it on the testing data with the same set of selected descriptive features. You will then calculate both the TSS and HSS scores and print them out. **Note:** The testing data has the samples in it that are below our threshold value, so you will first need to filter those out of the data you plan to pass to your model for testing. However, you still want those instances included in the calculation of the TSS and HSSS. So, your groud truth `lab` data should include all the instances in partition 2. You will need to concatenate a vector with all zeros in it to the match the labels you partitioned from the model testing data. 

Let's give you a representation of that:
    
    labels_from_data = [labels for samples > threshold] + [labels for samples <= threshold]
    predict_labels = [labels from the model on > thrshold samples] + [0s the length of samples <= threshold]



In [27]:
thresh = 0.5

kernel = ['linear', 'poly', 'rbf']
c_vals = [ 0.5, 1.0]
gamma_vals = [0.5, 1, 10]
temp = [kernel, c_vals, gamma_vals]
params = list(itertools.product(*temp))

In [28]:
#----------------------------------------------
for ind in range(len(selected_labels)):
     print(selected_labels[ind], "-_-_-_-_-_-_-"*5, end='\n\n')
     for kernel, c_value, gamma_value in params:
         classifier = SVC(class_weight = 'balanced', gamma=gamma_value, C=c_value, kernel=kernel)
         classifier.fit(di_train_x1[ind],di_train_y1[ind])
         y_pred = classifier.predict(di_test_x2[ind])
         tss_score = calc_tss(di_test_y2[ind], y_pred)
         hss_score = calc_hss(di_test_y2[ind], y_pred)
         print(f"TSS: {tss_score}")
         print(f"HSS: {hss_score}")
#----------------------------------------------

F-Val -_-_-_-_-_-_--_-_-_-_-_-_--_-_-_-_-_-_--_-_-_-_-_-_--_-_-_-_-_-_-

TN=48	FP=12	FN=2	TP=38
TSS: 0.75
HSS: 0.72
TN=48	FP=12	FN=2	TP=38
TSS: 0.75
HSS: 0.72
TN=48	FP=12	FN=2	TP=38
TSS: 0.75
HSS: 0.72
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=51	FP=9	FN=1	TP=39
TSS: 0.825
HSS: 0.7983870967741935
TN=51	FP=9	FN=1	TP=39
TSS: 0.825
HSS: 0.7983870967741935
TN=50	FP=10	FN=20	TP=20
TSS: 0.33333333333333337
HSS: 0.34782608695652173
TN=51	FP=9	FN=1	TP=39
TSS: 0.825
HSS: 0.7983870967741935
TN=49	FP=11	FN=1	TP=39
TSS: 0.7916666666666666
HSS: 0.76
TN=49	FP=11	FN=20	TP=20
TSS: 0.31666666666666665
HSS: 0.329004329004329
TN=47	FP=13	FN=1	TP=39
TSS: 0.7583333333333333
HSS: 0.7222222222222222
TN=46	FP=14	FN=1	TP=39
TSS: 0.7416666666666667
HSS: 0.7035573122529645
TN=49	FP=11	FN=1	TP=39
TSS: 0.7916666666666666
HSS: 0.76
TN=47	FP=13	

All of these results are getting unruely, we should maybe be saving them to do analysis on them too? Maybe I'll ask you to do that for the extra credit assignment.